In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np


In [5]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [6]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

driver.get("https://www.carrefour.fr/r?noRedirect=0")

page = driver.find_element_by_class_name('ds-carousel__items')
parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[2:13]

urls = []

for children in parent_urls:
    driver.get(children)
    page = driver.find_element_by_class_name('ds-carousel__items')
    children_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]

    for child in children_urls:
        driver.get(child)
        try:
            page = driver.find_element_by_class_name('ds-carousel__items')
            a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
            for url in a:
                urls.append(url)
        except:
            next
        
        
urls

['https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/agrumes',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/pommes-poires-et-raisins',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/bananes-et-kiwis',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-exotiques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-rouges',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/melons-et-pasteques',
 'https://www.carrefour.fr/r/fruits-et-legumes/fruits/prunes-peches-nectarines-abricots',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/legumes-de-saison',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/choux-poireaux-navets-et-celeris',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/carottes-et-pommes-de-terre',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/salades-et-tomates',
 'https://www.carrefour.fr/r/fruits-et-legumes/legumes/concombres-avocats-

In [8]:

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

product_pages = []

def scrape_shelters(urls):
    for link in urls:
        print(link)
        driver.get(link)
        sleep(1)
        shelter_main = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-3].text
        shelter_parent = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-2].text
        shelter_child = driver.find_element_by_class_name('breadcrumb-trail__list').find_elements_by_css_selector('li')[-1].text
        try:
            actions = ActionChains(driver)
            actions.move_to_element(driver.find_elements_by_css_selector("article")[-1]).perform()
        except:
            pass
        request_text = requests.get(link, headers=headers)   
        soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")  

        brands = []
        try:
            for label in driver.find_element_by_id('data_facet_marque').find_elements_by_css_selector("label"):
                brands.append(label.find_element_by_css_selector("span").text)
        except:
            pass
        product_pages.append({"url":link, 
                             "shelter_main": shelter_main,
                             "shelter_parent": shelter_parent,
                             "shelter_child": shelter_child,
                             "brands":brands,
                             "soup": soup.select('article')
                            })

scrape_shelters(urls)


https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-de-saison


/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:191: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


https://www.carrefour.fr/r/fruits-et-legumes/fruits/agrumes
https://www.carrefour.fr/r/fruits-et-legumes/fruits/pommes-poires-et-raisins
https://www.carrefour.fr/r/fruits-et-legumes/fruits/bananes-et-kiwis
https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-exotiques
https://www.carrefour.fr/r/fruits-et-legumes/fruits/fruits-rouges
https://www.carrefour.fr/r/fruits-et-legumes/fruits/melons-et-pasteques
https://www.carrefour.fr/r/fruits-et-legumes/fruits/prunes-peches-nectarines-abricots
https://www.carrefour.fr/r/fruits-et-legumes/legumes/legumes-de-saison
https://www.carrefour.fr/r/fruits-et-legumes/legumes/choux-poireaux-navets-et-celeris
https://www.carrefour.fr/r/fruits-et-legumes/legumes/carottes-et-pommes-de-terre
https://www.carrefour.fr/r/fruits-et-legumes/legumes/salades-et-tomates
https://www.carrefour.fr/r/fruits-et-legumes/legumes/concombres-avocats-betteraves-et-radis
https://www.carrefour.fr/r/fruits-et-legumes/legumes/champignons
https://www.carrefour.fr/r/fruits-

https://www.carrefour.fr/r/cremerie/beurres-et-margarines/beurres-doux
https://www.carrefour.fr/r/cremerie/beurres-et-margarines/beurres-demi-sel
https://www.carrefour.fr/r/cremerie/beurres-et-margarines/margarines-et-matieres-grasses
https://www.carrefour.fr/r/cremerie/cremes-fraiches-et-chantilly/epaisses
https://www.carrefour.fr/r/cremerie/cremes-fraiches-et-chantilly/fluides
https://www.carrefour.fr/r/cremerie/cremes-fraiches-et-chantilly/chantilly-et-cremes-anglaises
https://www.carrefour.fr/p/oeufs-gros-lustucru-3760150111571
https://www.carrefour.fr/p/oeufs-gros-lustucru-3760150111571
https://www.carrefour.fr/p/oeufs-tout-frais-lustucru-3760150111564
https://www.carrefour.fr/p/oeufs-tout-frais-lustucru-3760150111564
https://www.carrefour.fr/p/oeufs-bio-plein-air-lustucru-3760150111113
https://www.carrefour.fr/p/oeufs-bio-plein-air-lustucru-3760150111113
https://www.carrefour.fr/r/cremerie/oeufs#flag.common.organic
https://www.carrefour.fr/p/oeufs-bio-plein-air-lustucru-376015011

https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/camemberts-coulommiers-et-bries
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/roqueforts-et-bleus
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/chevres-et-brebis
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/emmental-comte-et-cantal
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/fromages-de-caractere
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/fromages-alleges
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/fromages-a-tartiner
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage/fromages-enfants
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage-pour-preparation/raclette-fondue-et-tartiflette
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage-pour-preparation/fromages-tranches
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage-pour-preparation/fromages-rapes
https://www.carrefour.fr/r/fromage-et-charcuterie/fromage-pour-preparation/f

https://www.carrefour.fr/r/traiteur/pates-fraiches-et-gnocchis#flag.common.fresh
https://www.carrefour.fr/p/pates-fraiches-tortellini-ricotta-epinard-rana-8001665704472
https://www.carrefour.fr/p/pates-fraiches-tortellini-ricotta-epinard-rana-8001665704472
https://www.carrefour.fr/r/traiteur/pates-fraiches-et-gnocchis#flag.common.fresh
https://www.carrefour.fr/p/pates-fraiches-tortellini-pesto-basilic-pignons-rana-8001665704489
https://www.carrefour.fr/p/pates-fraiches-tortellini-pesto-basilic-pignons-rana-8001665704489
https://www.carrefour.fr/r/traiteur/pates-fraiches-et-gnocchis#flag.common.fresh
https://www.carrefour.fr/p/pates-fraiches-tortellini-poulet-a-la-paysanne-rana-8001665704526
https://www.carrefour.fr/p/pates-fraiches-tortellini-poulet-a-la-paysanne-rana-8001665704526
https://www.carrefour.fr/r/traiteur/pates-fraiches-et-gnocchis#flag.common.fresh
https://www.carrefour.fr/p/pates-fraiches-cappelletti-jambon-cru-rana-8001665704533
https://www.carrefour.fr/p/pates-fraiches-

https://www.carrefour.fr/r/surgeles/pizzas#flag.common.frozen
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.organic
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.vegetarian
https://www.carrefour.fr/p/pizza-4-formaggi-carrefour-3560070124817
https://www.carrefour.fr/p/pizza-4-formaggi-carrefour-3560070124817
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.frozen
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.vegetarian
https://www.carrefour.fr/p/pizza-mozzarella-ristorante-4001724819806
https://www.carrefour.fr/p/pizza-mozzarella-ristorante-4001724819806
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.vegetarian
https://www.carrefour.fr/p/pizza-quattro-formaggi-ristorante-4001724818908
https://www.carrefour.fr/p/pizza-quattro-formaggi-ristorante-4001724818908
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.frozen
https://www.carrefour.fr/r/surgeles/pizzas#flag.common.vegetarian
https://www.carrefour.fr/p/pizza-tonno-ristorante-4001724819301

https://www.carrefour.fr/r/epicerie-sucree/boissons-chaudes/thes
https://www.carrefour.fr/r/epicerie-sucree/boissons-chaudes/infusions
https://www.carrefour.fr/r/epicerie-sucree/boissons-chaudes/chocolats-en-poudre
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/cereales-enfants
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/cereales-adultes
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/mueslis-et-avoines
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/barres-cereales
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/biscuits-petit-dejeuner
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/biscottes-pains-grilles-et-galettes
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/pain-de-mie
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/brioches-et-viennoiseries
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/confitures
https://www.carrefour.fr/r/epicerie-sucree/petit-dejeuner/pates-a-tartiner-et-miels
h

https://www.carrefour.fr/p/vin-rouge-bergerac-la-cave-d-augustin-florent-3245415191722
https://www.carrefour.fr/p/vin-rouge-vin-de-pays-d-oc-merlot-roche-mazet-3175520017276
https://www.carrefour.fr/p/vin-rouge-vin-de-pays-d-oc-merlot-roche-mazet-3175520017276
https://www.carrefour.fr/r/alcools-et-produits-aperitifs/cubis-de-vin#flag.common.igp
https://www.carrefour.fr/p/vin-rouge-cotes-du-rhone-les-pierrasques-cave-d-a-flor-3270190129363
https://www.carrefour.fr/p/vin-rouge-cotes-du-rhone-les-pierrasques-cave-d-a-flor-3270190129363
https://www.carrefour.fr/p/vin-rouge-cotes-du-rhone-cellier-des-dauphins-prestige-3179077062802
https://www.carrefour.fr/p/vin-rouge-cotes-du-rhone-cellier-des-dauphins-prestige-3179077062802
https://www.carrefour.fr/p/vin-rose-cotes-du-rhone-cellier-des-dauphins-prestige-3179071000343
https://www.carrefour.fr/p/vin-rose-cotes-du-rhone-cellier-des-dauphins-prestige-3179071000343
https://www.carrefour.fr/p/vin-blanc-i-g-p-pays-d-oc-autrement-bio-351412010835

In [9]:
data = pd.DataFrame(product_pages)
#data.to_csv('carrefour_catalog.csv')


In [10]:
pages = []
for index, row in data.iterrows():
    soup = row.soup
    pages.append({"shelter_main": row.shelter_main,
                  "shelter_parent": row.shelter_parent,
                  "shelter_child": row.shelter_child,                  
                  "brands": row.brands,
                  "page": soup
                })

In [11]:
products = []
for page in pages:
    for content in page['page'][1:-1]:
        name = content.find("a", class_="product-card-title--food")["title"]
        packaging = content.find("div", class_="product-card-packaging").text.strip()
        price = content.find("span", class_="product-card-price__price--final").text.strip()
        try:
            price_per_unit = content.find("div", class_="product-card-per-unit-label").text.strip()
        except:
            pass
#       promo_price = line[14:-1]
        try : 
            image_url = content.find("a", class_="product-card-image").find("img", class_="product-card-image__image")["data-src"]
        except:
            image_url = content.find("a", class_="product-card-image").find("img", class_="product-card-image__image")["src"]
        origin = content.find("div", class_="product-card__origin").text.strip() if content.find("div", class_="product-card__origin") else ""
        url = content.find("a", class_="product-card-title--food")["href"]
        ean = content.find("div", class_="add-to-shoppinglist")["ean"]
        offer = content.find("div", class_="ds-product-card--vertical-highlightlabel").text.strip()
        offer_description = content.find("span", class_="promotion-label__txt").text.strip() if content.find("span", class_="promotion-label__txt") else ""
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand in name])
        else:
            brand = ""
                
        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "description1": name, 
                         "description2": packaging,                        
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                        # "promo_price": promo_price,
                         "origin": origin,
                         "image_url": image_url,                    
                         "url": url,
                         "ean": ean,
                         "offer": offer,
                         "offer_description": offer_description
                        })


In [21]:
product_data = pd.DataFrame(products)
product_data = product_data.drop_duplicates()

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_carrefour.csv', index = None, header=True)


In [20]:
product_data.head()

,shelter_main,shelter_parent,shelter_child,description1,description2,brand,price,price_per_unit,origin,image_url,url,ean,offer,offer_description
0,Fruits et Légumes,Fruits,Fruits de saison,Citrons cultivés sans pesticide de synthèse dè...,le sachet d'1kg,CARREFOUR,"2,50€",2.50 € / Kilogramme,ORIGINE C.E.E.,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,/p/citrons-cultives-sans-pesticide-de-synthese...,3523680420670,Promotion,"2 achetés = 0,5€ de remise"
1,Fruits et Légumes,Fruits,Fruits de saison,Pommes Golden delicious FILIERE QUALITE CARREFOUR,la barquette de 6 fruits,CARREFOUR,"4,00€",3.12 € / Kilogramme,FRANCE,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,/p/pommes-golden-delicious-filiere-qualite-car...,3276555715950,,
2,Fruits et Légumes,Fruits,Fruits de saison,Citron bio,le filet de 500g,,"1,69€",3.38 € / Kilogramme,ESPAGNE,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,/p/citron-bio-3270190010272?t=R01F01SF01,3270190010272,Prime Bio,*Primes Fidélité: -10% si inscrit
3,Fruits et Légumes,Fruits,Fruits de saison,Poire Williams Bio CARREFOUR,la barquette de 4,CARREFOUR,"2,95€",2.95 € / Kilogramme,FRANCE,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,/p/poire-williams-bio-carrefour-3276550243588?...,3276550243588,Prime Bio,*Primes Fidélité: -10% si inscrit
4,Fruits et Légumes,Fruits,Fruits de saison,Poire Conférence Bio CONFERENCE,la barquette de 4 fruits,,"3,28€",3.28 € / Pièce,ORIGINE C.E.E.,/media/280x280/Photosite/PRODUITS_FRAIS_TRANSF...,/p/poire-conference-bio-conference-32765578765...,3276557876550,Prime Bio,*Primes Fidélité: -10% si inscrit


In [22]:

# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_main, shelter_parent, shelter_child, description1, description2, brand, price, price_per_unit, origin, image_url, url, ean, offer, offer_description]
Index: []
